In [ ]:
from google.colab import files
uploaded = files.upload()

Saving HR attrition.csv to HR attrition (1).csv


In [ ]:
import pandas as pd
A = pd.read_csv("HR attrition (1).csv")

In [ ]:
A.head(2)

,Attrition,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,41,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,0,49,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7


In [ ]:
A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Attrition                 1470 non-null   int64 
 1   Age                       1470 non-null   int64 
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [ ]:
def replacer(df):
    import pandas as pd
    Q = pd.DataFrame(df.isna().sum())
    Q.columns=["CT"]
    w = list(Q[Q.CT>0].index)
    cat = []
    con = []
    for i in w:
        if(df[i].dtypes=="object"):
            cat.append(i)
        else:
            con.append(i)
    for i in con:
        replacer = df[i].mean()
        df[i] = df[i].fillna(replacer)
    for i in cat:
        replacer = pd.DataFrame(df[i].value_counts()).index[0]
        df[i] = df[i].fillna(replacer)

In [ ]:
def preprocessing(df,removalcolumns,ycolumn):
    df = df.drop(labels=removalcolumns,axis=1)
    X = df.drop(labels=ycolumn,axis=1)
    Y = df[ycolumn]

    cat = []
    con = []
    for i in X.columns:
        if(X[i].dtypes=="object"):
            cat.append(i)
        else:
            con.append(i)

    from sklearn.preprocessing import MinMaxScaler
    mm = MinMaxScaler()
    X1 = pd.DataFrame(mm.fit_transform(X[con]),columns=con)
    X2 = pd.get_dummies(X[cat])
    X = X1.join(X2)

    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    Y = pd.DataFrame(le.fit_transform(Y),columns=Y.columns)

    from sklearn.model_selection import train_test_split
    xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=30)
    return xtrain,xtest,ytrain,ytest

In [ ]:
replacer(A)

In [ ]:
xtrain,xtest,ytrain,ytest=preprocessing(A,["Over18"],["Attrition"])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:

from keras import Sequential
nn = Sequential()
from keras.layers import Dense
nn.add(Dense(100,activation="relu",input_dim=len(xtrain.columns)))  #Input
nn.add(Dense(50,activation="relu")) #Hidden
nn.add(Dense(1,activation="sigmoid")) #Output
nn.compile(loss="binary_crossentropy",metrics="accuracy")
nn.fit(xtrain,ytrain,epochs=500,verbose=1)
pred_ts = nn.predict(xtest)
w1 = []
for i in pred_ts:
    if(i<0.5):
        w1.append(0)
    else:
        w1.append(1)

pred_tr = nn.predict(xtrain)
w2 = []
for i in pred_tr:
    if(i<0.5):
        w2.append(0)
    else:
        w2.append(1)


from sklearn.metrics import accuracy_score
acc_ts = accuracy_score(ytest,w1)

from sklearn.metrics import accuracy_score
acc_tr = accuracy_score(ytrain,w2)

if(acc_ts < acc_tr):
    print("Overfitting has occurred")

print("Training accuracy %.4f"%acc_tr)
print("Testing accuracy %.4f"%acc_ts)

Epoch 1/500
37/37 [==============================] - 1s 2ms/step - loss: 0.4224 - accuracy: 0.8342
Epoch 2/500
37/37 [==============================] - 0s 1ms/step - loss: 0.3688 - accuracy: 0.8401
Epoch 3/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3415 - accuracy: 0.8605
Epoch 4/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3272 - accuracy: 0.8690
Epoch 5/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3135 - accuracy: 0.8801
Epoch 6/500
37/37 [==============================] - 0s 2ms/step - loss: 0.3063 - accuracy: 0.8869
Epoch 7/500
37/37 [==============================] - 0s 2ms/step - loss: 0.2951 - accuracy: 0.8895
Epoch 8/500
37/37 [==============================] - 0s 2ms/step - loss: 0.2861 - accuracy: 0.8946
Epoch 9/500
37/37 [==============================] - 0s 2ms/step - loss: 0.2763 - accuracy: 0.8997
Epoch 10/500
37/37 [==============================] - 0s 2ms/step - loss: 0.2711 - accuracy: 0.8988
Epoch 11/